<a href="https://colab.research.google.com/github/naitotomoyuki/YoloLearning_with_ggl_colab/blob/main/yolo12s_Tray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os
import shutil

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [4]:
# --- 縦横90°切替を学習に混ぜるコールバック ---
import torch, random

def _rotate_xywh_norm(xywh, k):
    x, y, w, h = xywh.unbind(-1)
    if k % 4 == 0:      xn, yn, wn, hn = x,       y,       w, h
    elif k % 4 == 1:    xn, yn, wn, hn = 1.0 - y, x,       h, w   # +90° CCW
    elif k % 4 == 2:    xn, yn, wn, hn = 1.0 - x, 1.0 - y, w, h   # 180°
    else:               xn, yn, wn, hn = y,       1.0 - x, h, w   # 270°
    return torch.stack([xn, yn, wn, hn], dim=-1)

def rotate_batch_90deg_callback(trainer):
    # バッチ全体に 90°/270° を p で適用（縦⇄横の切替を学習させる）
    p = 0
    if random.random() > p:
        return
    b = trainer.batch
    imgs = b.get("img", None)
    if imgs is None or imgs.ndim != 4:
        return
    k = random.choice([1, 3])  # 90° or 270°
    b["img"] = torch.rot90(imgs, k=k, dims=[-2, -1])
    if b.get("bboxes", None) is not None and b["bboxes"].numel() > 0:
        b["bboxes"] = _rotate_xywh_norm(b["bboxes"], k)

callbacks = {"on_preprocess_batch": rotate_batch_90deg_callback}


In [ ]:
output_dir = "runs_experiment"  # runs_experimentが既に存在している場合削除
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print(f"✅ ディレクトリ '{output_dir}' を削除しました。")
else:
    print(f"⚠️ ディレクトリ '{output_dir}' は存在しません。")

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

In [ ]:
#!pip install -U torch torchvision torchaudio
!pip -q install ultralytics==8.3.177 --no-deps

In [ ]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

In [8]:
# YAML
import yaml

# YAMLファイルを読み込む
data_yaml = "/content/dataset/data.yaml"
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

data.yaml のパスを更新しました。


In [9]:
# yolo学習用関数の定義（Colab用）
from ultralytics import YOLO
import os

def train_yolo(
    model_path,
    data_yaml,
    epochs=200,
    batch_size=16,
    img_size=960,
    output_dir="runs_experiment",
    iou_nms=0.9,
    mosaic=0.15,
    mixup=0.05,
    copy_paste=0.6,
    close_mosaic=15,
    degrees=0.0, shear=0.0, perspective=0.0, translate=0.05, scale=0.2,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    erasing=0.1, fliplr=0.5, flipud=0.0,
    lr0=0.004,
    optimizer="AdamW",
    cos_lr=True,
    warmup_epochs=3,
    amp=True,
    workers=0,
    project_name="yolo_project",
    callbacks=callbacks,
    **kwargs,                       # ← ここが必須（末尾にカンマも）
    #seed = 42, #モデル比較時の乱数設定
):
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    os.makedirs(output_dir, exist_ok=True)
    print(f"Training YOLO with {epochs} epochs…")

    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,

        # 途中チェックポイントは作らない（save_periodを渡さない）
        # save=True のままでOK（best.pt & last.ptは出力される）

        # 学習率まわり
        optimizer=optimizer,
        lr0=lr0,
        cos_lr=cos_lr,
        warmup_epochs=warmup_epochs,

        # Aug/正則化
        mosaic=mosaic,
        mixup=mixup,
        copy_paste=copy_paste,
        close_mosaic=close_mosaic,
        degrees=degrees,
        shear=shear,
        perspective=perspective,
        translate=translate,
        scale=scale,
        hsv_h=hsv_h, hsv_s=hsv_s, hsv_v=hsv_v,
        erasing=erasing,
        fliplr=fliplr,
        flipud=flipud,

        # 検証時NMS
        iou=iou_nms,

        # 実行系
        amp=amp,
        workers=workers,
        project=output_dir,
        name=project_name,

        # 追加パラメータ（cacheやrect、patience等）はここから注入
        **kwargs,
        # callbacks=callbacks   # ← ここでRandomRotate90適用
    )

    print(f"Training complete. Results saved to {output_dir}/{project_name}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [12]:
from ultralytics import YOLO
# ====== 設定 ======
model_path = "/content/drive/MyDrive/yolo11n.pt"  # 初回学習
# model_path = "/content/drive/MyDrive/20250801iFujicco_item_yolo12m.pt"  # 継続学習の場合

data_yaml = "/content/dataset/data.yaml"
project_name = "yolo_learning"

train_params = {
    "model_path": model_path,
    "data_yaml": data_yaml,
    "epochs": 200,
    "batch_size": 256,
    "lr0": 0.004,
    "img_size": 640,
    "output_dir": "runs_experiment",
    "cache": "ram",
    "rect": True,
    "iou_nms": 0.60,
    "mosaic": 0.0,
    "mixup": 0.0,
    "copy_paste": 0.0,
    "close_mosaic": 0,
    "degrees": 90.0,
    "shear": 0.0,
    "perspective": 0.0,
    "translate": 0.03,
    "scale": 0.2,
    "hsv_h": 0.015,
    "hsv_s": 0.7,
    "hsv_v": 0.4,
    "erasing": 0.1,
    "fliplr": 0.5,
    "flipud": 0.5,
    "lr0": 0.004,
    "optimizer": "AdamW",
    "cos_lr": True,
    "warmup_epochs": 3,
    "amp": True,
    "workers": 2,
    "project_name": project_name,
    # "callbacks": callbacks # Callbacks are not a valid argument in this version
}

# ====== 実行 ======
train_yolo(**train_params)

Loading model from /content/drive/MyDrive/yolo11n.pt
Training YOLO with 200 epochs…
New https://pypi.org/project/ultralytics/8.3.178 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.177 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=0, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset/data.yaml, degrees=90.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.1, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.6, keras=False, kobj=1.0, line_width=None, lr0=0.004, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDriv

train: Scanning /content/dataset/labels/train.cache... 1179 images, 51 backgrounds, 0 corrupt: 100%|██████████| 1179/1179 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.8GB RAM): 100%|██████████| 1179/1179 [00:05<00:00, 207.25it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1632.3±1563.5 MB/s, size: 1934.5 KB)


...

      3/200      6.79G     0.8151      1.259      1.259         30        640: 100%|██████████| 19/19 [00:04<00:00,  4.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.93it/s]

                   all        337        360     0.0119       0.25     0.0109    0.00282



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/19 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [11]:
output_dir = "runs_experiment"
best_model_path = f"{output_dir}/{project_name}/weights/best.pt"
last_model_path = f"{output_dir}/{project_name}/weights/last.pt"

from google.colab import files
import os

for model_path in [best_model_path, last_model_path]:
    if os.path.exists(model_path):
        print(f"Downloading: {model_path}")
        files.download(model_path)
    else:
        print(f"[ERROR] {os.path.basename(model_path)} が見つかりません: {model_path}")



Downloading: runs_experiment/yolo_learning/weights/best.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloading: runs_experiment/yolo_learning/weights/last.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>